In [1]:
import pandas as pd
import numpy as np
import logging
import sys
from datetime import datetime

# a little hacky, but works if you don't want to actually install the
# custom packages 
sys.path.append('../')
from uniswapv3_simulator.pool import Uniswapv3Pool
from uniswapv3_simulator.utils import pool_init_price, solve_for_liquidity_delta
from uniswapv3_simulator.math import tick_to_sqrt_price, sqrt_price_to_tick
from utils import amount_to_float

In [2]:
POOL = 'FRAX-WETH-3000'
MAX_DATE = '2022-01-28'

timestamp = datetime.now().strftime('%y%m%d%H%M%S')
logging.basicConfig(level=logging.INFO,
                    filename=f'./logs/{POOL}_{timestamp}.log')
logging.getLogger('uniswap-v3').setLevel(logging.DEBUG)
logging.getLogger('covalent_api').setLevel(logging.DEBUG)

In [3]:
data = pd.read_pickle('../data/pool_data_clean.pickle')
swaps = data[POOL]['swaps']
liquidity = data[POOL]['liquidity']

In [4]:
swaps.info()
swaps.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype              
---  ------                          --------------  -----              
 0   swap_time                       22 non-null     datetime64[ns, UTC]
 1   tx_hash                         22 non-null     string             
 2   token_0_amount                  22 non-null     float64            
 3   token_1_amount                  22 non-null     float64            
 4   pool_address                    22 non-null     string             
 5   price_tick                      22 non-null     int64              
 6   price_tick_adjusted             22 non-null     float64            
 7   contract_address_token_0        22 non-null     string             
 8   contract_name_token_0           22 non-null     string             
 9   contract_ticker_symbol_token_0  22 non-null     string             
 10  contract_decimal

,swap_time,tx_hash,token_0_amount,token_1_amount,pool_address,price_tick,price_tick_adjusted,contract_address_token_0,contract_name_token_0,contract_ticker_symbol_token_0,contract_decimals_token_0,contract_address_token_1,contract_name_token_1,contract_ticker_symbol_token_1,contract_decimals_token_1,pool_fee,pool_tick_spacing,pool_deploy_time
0,2021-06-30 12:29:16+00:00,0xf748dd84e00e1c911304b99f9b16519cc2368536a689...,-30.518300,0.014412,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,-76573,0.000473,0x853d955acef822db058eb8505911ed77f175b99e,Frax,FRAX,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,3000,60,2021-05-06 03:25:59+00:00
1,2021-06-30 12:24:49+00:00,0xe23bbb2aa6eb31d8dd92206a586c653bad303c8c3577...,-24.459936,0.011464,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,-76656,0.000469,0x853d955acef822db058eb8505911ed77f175b99e,Frax,FRAX,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,3000,60,2021-05-06 03:25:59+00:00
2,2021-06-29 09:44:16+00:00,0xb2e143ef81308be550a0fa20ca108abacf5f864307f1...,84.305682,-0.039601,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,-76723,0.000466,0x853d955acef822db058eb8505911ed77f175b99e,Frax,FRAX,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,3000,60,2021-05-06 03:25:59+00:00
3,2021-06-29 00:48:38+00:00,0xd8a734393694ea6e9697ec7cad130662aa0082da073f...,214.492811,-0.105023,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,-76492,0.000477,0x853d955acef822db058eb8505911ed77f175b99e,Frax,FRAX,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,3000,60,2021-05-06 03:25:59+00:00
4,2021-06-28 04:49:57+00:00,0x46d5c298c471cbce6dfd3493ce382feef80888b6db08...,180.440263,-0.093429,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,-75893,0.000506,0x853d955acef822db058eb8505911ed77f175b99e,Frax,FRAX,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,3000,60,2021-05-06 03:25:59+00:00


In [5]:
liquidity.info()
liquidity.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype              
---  ------                          --------------  -----              
 0   txn_time                        10 non-null     datetime64[ns, UTC]
 1   tx_hash                         10 non-null     string             
 2   token_0_amount                  10 non-null     float64            
 3   token_1_amount                  10 non-null     float64            
 4   pool_address                    10 non-null     string             
 5   liquidity_event                 10 non-null     string             
 6   price_tick_lower                10 non-null     int64              
 7   price_tick_upper                10 non-null     int64              
 8   price_tick_lower_adjusted       10 non-null     float64            
 9   price_tick_upper_adjusted       10 non-null     float64            
 10  contract_address_

,txn_time,tx_hash,token_0_amount,token_1_amount,pool_address,liquidity_event,price_tick_lower,price_tick_upper,price_tick_lower_adjusted,price_tick_upper_adjusted,...,contract_decimals_token_0,contract_address_token_1,contract_name_token_1,contract_ticker_symbol_token_1,contract_decimals_token_1,event_name,pool_fee,pool_tick_spacing,pool_deploy_time,liquidity
0,2021-07-09 09:36:17+00:00,0x91dbaf36e40403f996999185adc9a79da96ce550a755...,14642.824315,10.014862,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,REMOVE_LIQUIDITY,-77640,-75840,0.000425,0.000509,...,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,DecreaseLiquidity,3000,60,2021-05-06 03:25:59+00:00,-8858.679599
1,2021-07-09 08:25:15+00:00,0xc4d2194e88eec2f2658b56817a6afdd009f77cd07a73...,14642.824315,10.014862,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,ADD_LIQUIDITY,-77640,-75840,0.000425,0.000509,...,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,Mint,3000,60,2021-05-06 03:25:59+00:00,8858.679599
2,2021-07-01 18:19:17+00:00,0xcad737c6e153ebd1021a611659bff443eada6a607182...,1790.080527,1.204647,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,REMOVE_LIQUIDITY,-85200,-70920,0.000200,0.000832,...,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,Burn,3000,60,2021-05-06 03:25:59+00:00,-158.115676
3,2021-06-25 06:19:32+00:00,0xe731a45b47b1e949f913b5431b58a9060147651a6d98...,1533.838996,0.695150,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,REMOVE_LIQUIDITY,-77220,-74640,0.000443,0.000574,...,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,DecreaseLiquidity,3000,60,2021-05-06 03:25:59+00:00,-523.478348
4,2021-06-24 00:53:00+00:00,0xa0e7b1a7a85a7814b8de614c9f1fc41df7e2994da348...,1222.031010,0.853419,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,ADD_LIQUIDITY,-77220,-74640,0.000443,0.000574,...,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,IncreaseLiquidity,3000,60,2021-05-06 03:25:59+00:00,523.478348


In [6]:
adds = liquidity.loc[liquidity['liquidity_event'] == 'ADD_LIQUIDITY', :].copy()
adds = adds.sort_values('txn_time').reset_index(drop=True)
adds.head()

,txn_time,tx_hash,token_0_amount,token_1_amount,pool_address,liquidity_event,price_tick_lower,price_tick_upper,price_tick_lower_adjusted,price_tick_upper_adjusted,...,contract_decimals_token_0,contract_address_token_1,contract_name_token_1,contract_ticker_symbol_token_1,contract_decimals_token_1,event_name,pool_fee,pool_tick_spacing,pool_deploy_time,liquidity
0,2021-05-06 03:25:59+00:00,0xf803bec9942ee16a73f4f5fc861eb642f4f88a98e72b...,388.042314,0.181108,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,ADD_LIQUIDITY,-108180,-76020,0.000020,0.000500,...,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,IncreaseLiquidity,3000,60,2021-05-06 03:25:59+00:00,17.365023
1,2021-06-23 12:40:10+00:00,0xea79367dffdb76a41d0137f453895fabc4746eb1691a...,2250.310532,1.000000,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,ADD_LIQUIDITY,-85200,-70920,0.000200,0.000832,...,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,IncreaseLiquidity,3000,60,2021-05-06 03:25:59+00:00,158.115676
2,2021-06-23 12:44:50+00:00,0xc1782ac45b1c105a4cc0bf0a4edfffe4fb139cb7b8a5...,118348.423382,58.504101,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,ADD_LIQUIDITY,-82920,-69060,0.000251,0.001002,...,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,Mint,3000,60,2021-05-06 03:25:59+00:00,8986.024272
3,2021-06-24 00:53:00+00:00,0xa0e7b1a7a85a7814b8de614c9f1fc41df7e2994da348...,1222.031010,0.853419,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,ADD_LIQUIDITY,-77220,-74640,0.000443,0.000574,...,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,IncreaseLiquidity,3000,60,2021-05-06 03:25:59+00:00,523.478348
4,2021-07-09 08:25:15+00:00,0xc4d2194e88eec2f2658b56817a6afdd009f77cd07a73...,14642.824315,10.014862,0x92c7b5ce4cb0e5483f3365c1449f21578ee9f21a,ADD_LIQUIDITY,-77640,-75840,0.000425,0.000509,...,18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,Wrapped Ether,WETH,18,Mint,3000,60,2021-05-06 03:25:59+00:00,8858.679599


In [7]:
first_add_hash = adds.at[0, 'tx_hash']
print(f'First liquidity add hash: {first_add_hash}')

First liquidity add hash: 0xf803bec9942ee16a73f4f5fc861eb642f4f88a98e72b9dbcfc2a1613bbf282d9


In [8]:
# from https://etherscan.io/tx/0xf803bec9942ee16a73f4f5fc861eb642f4f88a98e72b9dbcfc2a1613bbf282d9#eventlog
liquidity_delta = amount_to_float('17365023473069769310', 18)  # belive all liquidity amounts use 18 decimals
assert liquidity_delta == adds.at[0, 'liquidity']

token0 = adds.at[0, 'token_0_amount']
token1 = adds.at[0, 'token_1_amount']
tick_lower = adds.at[0, 'price_tick_lower']
tick_upper = adds.at[0, 'price_tick_upper']

init_price = pool_init_price(token0, token1, tick_upper, tick_lower, liquidity_delta)

token0_symb = liquidity.at[0, 'contract_ticker_symbol_token_0']
token1_symb = liquidity.at[0, 'contract_ticker_symbol_token_1']
print(f'Pool initial price ({token1_symb}/{token0_symb}): {init_price:,.8f}')
print(f'Pool initial price ({token0_symb}/{token1_symb}): {1 / init_price:,.4f}')

Pool initial price (WETH/FRAX): 0.00022221
Pool initial price (FRAX/WETH): 4,500.2103


In [9]:
cols = ['tx_hash', 'txn_time', 'liquidity_event']
liqu_txn = liquidity.loc[:, cols].copy()
liqu_txn.reset_index(drop=False, inplace=True)
liqu_txn.rename(columns={'liquidity_event': 'event', 'index': 'orig_idx'}, inplace=True)

cols = ['tx_hash', 'swap_time']
swap_txn = swaps.loc[:, cols].copy()
swap_txn.reset_index(drop=False, inplace=True)
swap_txn.rename(columns={'swap_time': 'txn_time', 'index': 'orig_idx'}, inplace=True)
swap_txn['event'] = 'SWAP'

all_txn = pd.concat([liqu_txn, swap_txn], axis=0)
all_txn = all_txn.sort_values('txn_time').reset_index(drop=True)
all_txn.drop(all_txn.index[all_txn['txn_time'] > MAX_DATE], axis=0, inplace=True)

all_txn

,orig_idx,tx_hash,txn_time,event
0,9,0xf803bec9942ee16a73f4f5fc861eb642f4f88a98e72b...,2021-05-06 03:25:59+00:00,ADD_LIQUIDITY
1,21,0x3bb9b28932d71971f9cb9b2c5be3beaa861e3a97b3fe...,2021-05-19 10:37:08+00:00,SWAP
2,20,0xd609243372165441ee59ef9ccfcf1a4e47312d0da973...,2021-05-22 21:45:49+00:00,SWAP
3,19,0x0976f4ead64a7ed32a113d8c86393e76ad6fa83cf266...,2021-06-04 13:26:01+00:00,SWAP
4,18,0xd1f2bb1e6fcc28679ee31d9c04d64221cd720f9faecc...,2021-06-07 04:48:50+00:00,SWAP
5,17,0x75f39ba11365235422c9ac292777bf94f040d80ceeab...,2021-06-09 08:36:36+00:00,SWAP
6,16,0xfcd534591b9c44fde1667dc796138a85efec5675135e...,2021-06-17 20:33:23+00:00,SWAP
7,8,0x8c0ad82afd257c5301c1815df3b590b6f6611300875a...,2021-06-18 09:01:28+00:00,REMOVE_LIQUIDITY
8,7,0xea79367dffdb76a41d0137f453895fabc4746eb1691a...,2021-06-23 12:40:10+00:00,ADD_LIQUIDITY
9,15,0x8916d0cbaaf78d49f6d462d7b7abf61f44bc19382cf9...,2021-06-23 12:40:10+00:00,SWAP


In [10]:
CHECKS_ON = True
# need to think about appropriate error tolerances
# TODO: maybe base these tolerances on the average transaction size?
TOKEN0_TOLS = {'atol': 1e-12, 'rtol': 1e-8}
TOKEN1_TOLS = {'atol': 1e-12, 'rtol': 1e-8}
LIQUIDITY_TOLS = {'atol': 1e-8, 'rtol': 1e-5}

In [11]:
fee = liquidity.at[0, 'pool_fee'] / 1e+6
tick_spacing = liquidity.at[0, 'pool_tick_spacing']
pool = Uniswapv3Pool(fee, tick_spacing, init_price)
print(f'{pool}')

position_id = 'LP123'
results = []
for i, row in all_txn.iterrows():
    logging.info(f'Transaction {i}.')
    txn = row['event']
    idx = row['orig_idx']
    
    if 'LIQUIDITY' in txn:
        token0 = liquidity.at[idx, 'token_0_amount']
        token1 = liquidity.at[idx, 'token_1_amount']
        if txn == 'REMOVE_LIQUIDITY':
            token0 = -1 * token0
            token1 = -1 * token1
        
        tick_lower = liquidity.at[idx, 'price_tick_lower']
        tick_upper = liquidity.at[idx, 'price_tick_upper']
        liquidity_delta = liquidity.at[idx, 'liquidity']
        
        if pd.isnull(liquidity_delta):
            liquidity_delta = solve_for_liquidity_delta(
                token0, 
                token1, 
                tick_lower, 
                tick_upper, 
                pool.sqrt_price
            )
        elif CHECKS_ON:
            ld_calc = solve_for_liquidity_delta(
                token0, 
                token1, 
                tick_lower, 
                tick_upper, 
                pool.sqrt_price
            )
            assert np.isclose(liquidity_delta, ld_calc, **LIQUIDITY_TOLS), (
                f'Calculated liquidity_delta {ld_calc:,.2f} does '
                f'not match liquidity_delta per the data {liquidity_delta:,.2f}.'
            )
            
        position = pool.position_map[(position_id, tick_lower, tick_upper)]
        # If the liquidity_delta is very, very close to the position's total
        # liquidity, set liquidity_delta to the total liquidity to completely
        # close out the position
        if np.isclose(-position.liquidity, liquidity_delta):
            liquidity_delta = -position.liquidity
        # we also make sure that liquidity_delta cannot be less than the 
        # position's total liquidity
        if liquidity_delta < 0:
            liquidity_delta = max(liquidity_delta, -position.liquidity)
                
        token0_calc, token1_calc = pool.set_position(
            position_id, 
            tick_lower,
            tick_upper,
            liquidity_delta
        )
    elif txn == 'SWAP':
        token0 = swaps.at[idx, 'token_0_amount']
        token1 = swaps.at[idx, 'token_1_amount']
        
        token = 0 if token0 > 0 else 1
        tokens_in = token0 if token == 0 else token1
        
        token0_calc, token1_calc = pool.swap(token, tokens_in)
    else:
        raise ValueError(f'{txn} is not a valid transaction type.')
    
    if CHECKS_ON:
        assert np.isclose(token0, -token0_calc, **TOKEN0_TOLS), (
            f'Transaction {i:,}: token0 output {-token0_calc:,.4f} does not '
            f'match token0 in the data {token0:,.4f}.'
        )
        assert np.isclose(token1, -token1_calc, **TOKEN1_TOLS), (
            f'Transaction {i:,}: token1 output {-token1_calc:,.4f} does not '
            f'match token1 in the data {token1:,.4f}.'
        )
    
    results.append({
        'sqrt_price': pool.sqrt_price, 
        'liquidity': pool.liquidity
    })
    print(f'Completed transaction {i}.')

Pool(price=0.0002, liquidity=0.00, fee=0.30%)
Completed transaction 0.
Completed transaction 1.
Completed transaction 2.
Completed transaction 3.
Completed transaction 4.
Completed transaction 5.
Completed transaction 6.
Completed transaction 7.
Completed transaction 8.
Completed transaction 9.
Completed transaction 10.
Completed transaction 11.
Completed transaction 12.
Completed transaction 13.
Completed transaction 14.
Completed transaction 15.
Completed transaction 16.
Completed transaction 17.
Completed transaction 18.
Completed transaction 19.
Completed transaction 20.
Completed transaction 21.
Completed transaction 22.
Completed transaction 23.
Completed transaction 24.
Completed transaction 25.
Completed transaction 26.
Completed transaction 27.
Completed transaction 28.
Completed transaction 29.
Completed transaction 30.
Completed transaction 31.
